# SBTi-Finance Tool - Check Temperature Scores from Data Provider
This notebook can be used to compare the temperature scores delivered from data providers with the scores calculated by the SBTi's version of the tool.

Please see the [methodology](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf), [guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/10/Financial-Sector-Science-Based-Targets-Guidance-Pilot-Version.pdf) and the [technical documentation](https://sciencebasedtargets.github.io/SBTi-finance-tool/) for more details. 

See 1_analysis_example (on [Colab](https://colab.research.google.com/github/OFBDABV/SBTi/blob/master/examples/1_analysis_example.ipynb) or [Github](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/master/examples/1_analysis_example.ipynb)) for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 

### Install the SBTi Python module
This installs the special branch from github that is used for testing purposes by the SBTi finance team.

In [1]:
%pip install SBTi-finance-tool

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
import pandas as pd

## Create the data directory
This is where the data files are to be placed before continuing with the calculations. Please check out to the [Data Requirements section](https://sciencebasedtargets.github.io/SBTi/DataRequirements.html) of the technical documentation for more details on data requirements and formatting. 


In [22]:
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/full_data_provider_.xlsx"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/Test_SP_scores/examples/data/full_data_provider.xlsx", "data/full_data_provider.xlsx")
if not os.path.isfile("data/testing_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/Test_SP_scores/examples/data/testing_portfolio.csv", "data/testing_portfolio.csv")
if not os.path.isfile("data/SP_temperature.csv"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/Test_SP_scores/examples/data/SP_temperature.csv", "data/SP_temperature.csv")

## Upload data files
If your are using your own files and not the example colab file imported aboove, now is the time to upload your data files into the /data directory

You need to provide three files:
1) The original data provider file (xlsx format)
2) The portfolio (csv)
3) A file with the service provider's temperature scores (csv). 

Name the files:
1) full_data_provider.xlsx
2) testing_portfolio.csv
3) SP_temperature.csv



##### Logging
The SBTi module uses the Python standard library logging utilities to send log messages. The log level can be changed according to the user's needs.

In [23]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

## Create a data provider
If using Colab (or Jupyter) just upload the file to the data directory. Otherwise you can provicde the full path to the file in the following cell.

In [24]:
provider = ExcelProvider(path="data/BBG_validation3/full_data_provider.xlsx")

## Load your portfolio
In our case the portfolio is stored as a CSV file. The portfolio should at least have an "id" (the identifier of the company) and a "proportion" (the weight of the company in your portfolio e.g. the value of the shares you hold) column.

Please see the technical documentation on [Data Legends](https://sciencebasedtargets.github.io/SBTi/Legends.html#) for details on data requirements. 

In [25]:
df_portfolio = pd.read_csv("data/BBG_validation3/portfolio_BBGTR20220823.csv", encoding="iso-8859-1")

In [26]:
df_portfolio.head(5)

,company_id,company_isin,company_name,investment_value,engagement_target
0,000050CH,CNE000000HT1,TIANMA MICROELECTRONICS-A,1000,False
1,000210KS,KR7000210005,DL HOLDINGS CO LTD,1000,False
2,000270KS,KR7000270009,KIA CORP,1000,False
3,000559CH,CNE000000FB3,WANXIANG QIANCHAO CO LTD-A,1000,False
4,000660KS,KR7000660001,SK HYNIX INC,1000,False


To load the data from the data provider, we have to pass a list of IPortfolioCompany instances. The module has a strict [data model](https://sciencebasedtargets.github.io/SBTi-finance-tool/autoapi/SBTi/interfaces/index.html) to convert Pandas Dataframe to the right object types we supplied a utility function.


In [27]:
companies = SBTi.utils.dataframe_to_portfolio(df_portfolio)

In [28]:
companies[1]

PortfolioCompany(company_name='DL HOLDINGS CO LTD', company_id='000210KS', company_isin='KR7000210005', investment_value=1000.0, engagement_target=False, user_fields=None)

## Calculate the temperature scores
In the amended portfolio you'll find your original portfolio, amended with the emissions and the temperature score.
Note that these are the reference temperatures that we will check the SO against.

In [29]:
temperature_score = TemperatureScore(                  # all available options:
    time_frames=list(SBTi.interfaces.ETimeFrames),     # ETimeFrames: SHORT MID and LONG
    scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3],    # EScopes: S3, S1S2 and S1S2S3
    aggregation_method=PortfolioAggregationMethod.WATS # Options for the aggregation method are WATS, TETS, AOTS, MOTS, EOTS, ECOTS, and ROTS.
)
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)

/home/mountainrambler/.local/lib/python3.8/site-packages/SBTi/target_validation.py:303: PerformanceWarning: indexing past lexsort depth may impact performance.
  :param target_columns: The columns that need to be returned


For every company the tool assigns a score for all the requested timeframe and scope combinations. In this example we used the full set resulting in 9 scores per company as displayed below:

In [31]:
#amended_portfolio[['company_name', 'time_frame', 'scope', 'temperature_score']].head(9)
pd.set_option("display.max_columns",None)

amended_portfolio.head(9)

,company_id,time_frame,scope,target_type,intensity_metric,coverage_s1,coverage_s2,coverage_s3,reduction_ambition,base_year,base_year_ghg_s1,base_year_ghg_s2,base_year_ghg_s3,start_year,end_year,achieved_reduction,company_name,isic,ghg_s1s2,ghg_s3,country,region,sector,industry_level_1,industry_level_2,industry_level_3,industry_level_4,company_revenue,company_market_cap,company_enterprise_value,company_total_assets,company_cash_equivalents,sbti_validated,company_isin,investment_value,engagement_target,sr15,annual_reduction_rate,slope,samplesize,model,variable,param,intercept,r2,temperature_score,temperature_results
0,AMDUS,SHORT,S1S2,Absolute,nan,0.95,0.95,NaN,0.2,2014.0,55120.998383,145832.000732,5.132210e+06,2014.0,2020.0,NaN,Advanced Micro Devices Inc,H50,32250.999451,5.554650e+06,UNITED STATES,None,None,Technology,Tech Hardware & Semiconductors,Semiconductors,Semiconductor Devices,16434.0,162771.976347,159978.976347,12419.0,2535.0,True,US0079031078,1000.0,False,Emissions|Kyoto Gases,0.033333,slope5,128.0,4.0,Emissions|Kyoto Gases,-0.212692,2.574303,0.609039,1.87,0.000000
1,AMDUS,SHORT,S3,absolute,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Advanced Micro Devices Inc,H50,32250.999451,5.554650e+06,UNITED STATES,None,None,Technology,Tech Hardware & Semiconductors,Semiconductors,Semiconductor Devices,16434.0,162771.976347,159978.976347,12419.0,2535.0,True,US0079031078,1000.0,False,Emissions|Kyoto Gases,NaN,slope5,128.0,4.0,Emissions|Kyoto Gases,-0.212692,2.574303,0.609039,3.20,1.000000
2,AMDUS,SHORT,S1S2S3,absolute,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Advanced Micro Devices Inc,H50,32250.999451,5.554650e+06,UNITED STATES,None,None,Technology,Tech Hardware & Semiconductors,Semiconductors,Semiconductor Devices,16434.0,162771.976347,159978.976347,12419.0,2535.0,True,US0079031078,1000.0,False,None,NaN,slope5,NaN,NaN,NaN,NaN,NaN,NaN,3.19,0.994227
3,AMDUS,MID,S1S2,absolute,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Advanced Micro Devices Inc,H50,32250.999451,5.554650e+06,UNITED STATES,None,None,Technology,Tech Hardware & Semiconductors,Semiconductors,Semiconductor Devices,16434.0,162771.976347,159978.976347,12419.0,2535.0,True,US0079031078,1000.0,False,Emissions|Kyoto Gases,NaN,slope15,128.0,4.0,Emissions|Kyoto Gases,-0.312329,2.697261,0.829523,3.20,1.000000
4,AMDUS,MID,S3,absolute,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Advanced Micro Devices Inc,H50,32250.999451,5.554650e+06,UNITED STATES,None,None,Technology,Tech Hardware & Semiconductors,Semiconductors,Semiconductor Devices,16434.0,162771.976347,159978.976347,12419.0,2535.0,True,US0079031078,1000.0,False,Emissions|Kyoto Gases,NaN,slope15,128.0,4.0,Emissions|Kyoto Gases,-0.312329,2.697261,0.829523,3.20,1.000000
5,AMDUS,MID,S1S2S3,absolute,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Advanced Micro Devices Inc,H50,32250.999451,5.554650e+06,UNITED STATES,None,None,Technology,Tech Hardware & Semiconductors,Semiconductors,Semiconductor Devices,16434.0,162771.976347,159978.976347,12419.0,2535.0,True,US0079031078,1000.0,False,None,NaN,slope15,NaN,NaN,NaN,NaN,NaN,NaN,3.20,1.000000
6,AMDUS,LONG,S1S2,absolute,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Advanced Micro Devices Inc,H50,32250.999451,5.554650e+06,UNITED STATES,None,None,Technology,Tech Hardware & Semiconductors,Semiconductors,Semiconductor Devices,16434.0,162771.976347,159978.976347,12419.0,2535.0,True,US0079031078,1000.0,False,Emissions|Kyoto Gases,NaN,slope30,128.0,4.0,Emissions|Kyoto Gases,-0.483331,2.841338,0.925768,3.20,1.000000
7,AMDUS,LONG,S3,absolute,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Advanced Micro Devices Inc,H50,32250.999451,5.554650e+06,UNITED STATES,None,None,Technology,Tech Hardware & Semiconductors,Semiconductors,Semiconductor Devices,16434.0,162771.976347,159978.976347,12419.0,2535.0,True,US0079031078,1000.0,False,Emissions|Kyoto Gases,NaN,slope30,128.0,4.0,Emissions|Kyoto Gases,-0.483331,2.841338,0.925768,3.20,1.000000
8,AMDUS,LONG,S1S2S3,absolute,None,NaN,

In [ ]:
#Make a dictionary with id, time-frame and scope as key (in a tuple), and temperature as value
reference_dict = {}

for index, row in amended_portfolio.iterrows():
    key = (row['company_id'], row['time_frame'].name, row['scope'].name)
    value = row['temperature_score']
    assert isinstance(value, float), f"not a number {key}"
    reference_dict[key] = value

In [ ]:
for i in range (len(amended_portfolio)):
    if i < 10:
        comp_id = amended_portfolio.loc[i]['company_id']
        time_fr = amended_portfolio.loc[i]['time_frame']
        scope = amended_portfolio.loc[i]['scope']
        amended_portfolio.at[i]['temperature_results'] = 99.2
    else:
        break

In [ ]:
amended_portfolio.head(9)

In [ ]:
myrow = amended_portfolio[(amended_portfolio['company_id'] == 'AMDUS') & (amended_portfolio['time_frame'] == 'SHORT') & (amended_portfolio['scope'] == 'S1S2')]
        

In [ ]:
#print(myrow.at[0,'temperature_score'])
myrow

In [ ]:
i = 0
for index, row in amended_portfolio.iterrows():
    if i < 10:
        key = (row['company_id'], row['time_frame'].name, row['scope'].name)
        value = row['temperature_score']
        amended_portfolio.loc[index, 'temperature_results'] = value + 5
        print(key, value, row['temperature_results'])
        i += 1
        

### Now read the file with service provider temperature scores
Select the approriate separator (some instances of excel save with semicolons instead of commas)

In [ ]:
#df_temps = pd.read_csv("data/BBG_validation/SP_temperature.csv", sep=";", encoding="ascii") #if the file has semicolons as separators
df_temps = pd.read_csv("data/BBG_validation3/SP_temp5.csv", encoding="iso-8859-1") #default is commas

### Check that the file looks OK

In [ ]:
df_temps.head(9)

### Make a dictionary with service providers temperatures

In [ ]:
SP_temperature_dict = {}

for index, row in df_temps.iterrows():
    scope = row['scope']
    #sometimes the way the scope is expressed is different, so change it to the standard form
    if scope == 'S12':
        scope = 'S1S2'
    elif scope == "S123":
        scope = 'S1S2S3'
    key = (row['company_id'], row['time_frame'], scope)
    value = row['temperature_score']
    assert isinstance(value, float), f"not a number {key}"
    SP_temperature_dict[key] = value
    
#only print using small test files if something has gone wrong   
#print(SP_temperature_dict)    

### Now add the SP temperature to the reference temperature in a list for each key

In [ ]:
for key, value in reference_dict.items():
    if key in SP_temperature_dict:
        reference_dict[key] = [value, SP_temperature_dict[key]]
    else:
        reference_dict[key] = "pass"

#print(reference_dict)
    

## Calculate temperature differences

The next cell calculates the differences in temperature scores from the two sources.
The user can set a threshold value for reported differences, this threshold is some number 0 to x. Default is 0.2 degrees. 


In [ ]:
#set the threshold value
allowed_temp_diff = 0.2

In [ ]:
for key, value in reference_dict.items():
    print(key)
    print(round(abs(value[0] - value[1]),2))

In [ ]:

temp_miss = {} #Empty dict to store reported deviances
num_pass = 0 # store the number of cases where SP temp is missing
num_temps = 0 # store the number of temperatures we look at
sum_diff = 0.0 # store the sum of deviations so we can do some calculations later

for key, value in reference_dict.items():
    num_temps += 1
    if value != 'pass':
        diff = abs(value[0] - value[1])
        sum_diff += diff
    else:
        num_pass += 1
    if diff > allowed_temp_diff:
        temp_miss[key] = value
print(f'The number of temperatures is {num_temps} and the average deviation is {round(sum_diff/num_temps, 2)} degrees.')
print(f'The number of cases where temperatures deviate more than {allowed_temp_diff} degrees is {len(temp_miss)}')
print(f'The number of temperatures where there is no SP temperature is {num_pass}.')            

In [ ]:
len(temp_miss)

### Make a dataframe from the temperature difference and export to a csv file

In [ ]:
df_output = pd.DataFrame.from_dict(temp_miss, orient = 'index', columns = ['ref temp', 'SP temp'])

In [ ]:
df_output.to_csv("data/BBG_validation3/temp_compare.csv")

In [ ]:
df_output = pd.DataFrame.from_dict(reference_dict, orient = 'index', columns = ['ref temp', 'SP temp'])

In [ ]:
my_port = amended_portfolio.set_index([('company_id', 'time_frame', 'scope')])

In [ ]:
lll = list(my_port.index.values)

In [ ]:
my_port.head()

In [ ]:
diff_index = []
for i in lll:
    new_index = (i[0], i[1].name, i[1].name)
    diff_index.append(new_index)

In [ ]:
full_diff = {}
x = 0
for key in diff_index:
    if key in my_port:
        x += 1
print(x)

In [ ]:
diff_dict = {}
for i in range(len(my_port)):
    name = my_port.index[i][0]
    timeframe = my_port.index[0][1].name
    scope = my_port.index[0][2].name
    diff_dict[(name, timeframe,scope)] = list(my_port.iloc[i][0:43])

In [ ]:
diff_with_targets = {}
for key, value in temp_miss.items():
    if key in diff_dict:
        diff_with_targets[key] = (value, diff_dict[key])
        

In [ ]:
len(diff_with_targets)

In [ ]:
df_diff_with_targets = pd.DataFrame.from_dict(diff_with_targets, orient='index')

In [ ]:
df_diff_with_targets.to_csv("data/BBG_validation3/temp_compare_targets.csv")